In [70]:
from src import API,Preprocessor
from src.preprocessing import remove_duplicates_Cleaner,moving_average_Filter,moving_max_Filter,moving_max_averaged_Filter,blank_Filter
import pandas as pd
from src.visualization import rssi_viewer

In [71]:

#API call
callApi=API()
start="2023-07-11 15:45:00.000000"
end="2023-07-11 16:05:00.000000"
raw_data=callApi.getRawDataForCartoWear('C1598FF23480',pd.to_datetime(start),pd.to_datetime(end))

#Preprocessing
sampling_time=2
window='5S'
preprocessor=Preprocessor(raw_data,sampling_time=sampling_time)
rssi_df=preprocessor.set_cleaner(remove_duplicates_Cleaner())\
        .set_filter(moving_max_averaged_Filter(window))\
        .process()


INFO - data Cleaning
INFO - cleaned, sorting
INFO - sorted, filtering


INFO - filtered


In [72]:
fig=rssi_viewer(rssi_df)


In [73]:
def get_strongest_rssi(df,timestamp,timedelta,n):
    """
    Return the n strongest rssi values at a given timestamp
    """
    start=timestamp
    end=timestamp+pd.Timedelta(seconds=timedelta)
    return df.query('@start<=timestamp<@end').sort_values('rssi',ascending=False)['macModule'].head(n)

In [74]:
#ici on est obligé de passer de l'heure locale à l'heure UTC pour se raccorder à la bdd
import logging
logger = logging.getLogger('cartobat')

min_time=pd.to_datetime(start)-pd.Timedelta(hours=2)+pd.Timedelta(seconds=timedelta)#permits to not have the issue of the starting data is incomplete
max_time=pd.to_datetime(end)-pd.Timedelta(hours=2)

time_range=pd.date_range(min_time,max_time,freq=pd.Timedelta(seconds=sampling_time))
number_of_mac_module=3


static_mac_module_list=get_strongest_rssi(rssi_df,min_time,timedelta,number_of_mac_module).to_list()
print(static_mac_module_list)



['A8032A311F6A', 'A8032A311F6A']
['A8032A311F6A', 'A8032A311F6A'] ['A8032A311F6A', 'A8032A311F6A']
['A8032A311F6A', 'A8032A311F6A'] ['A8032A311F6A']
['A8032A311F6A', 'A8032A311F6A'] ['A8032A311F6A']
['A8032A311F6A', 'A8032A311F6A'] ['A8032A311F6A']
['A8032A311F6A', 'A8032A311F6A'] ['A8032A311F6A']
['A8032A311F6A', 'A8032A311F6A', 'A8032A311DAA'] ['A8032A311F6A', 'A8032A311F6A', 'A8032A311DAA']


INFO - different mac_module detected at 2023-07-11 13:46:06
INFO - different mac_module detected at 2023-07-11 13:46:08
INFO - more than 2 different mac_module detected at 2023-07-11 13:46:12
INFO - static mac module set: ['A8032A311F96', 'A8032A31204A']
INFO - different mac_module detected at 2023-07-11 13:46:24
INFO - different mac_module detected at 2023-07-11 13:46:26
INFO - different mac_module detected at 2023-07-11 13:46:34


['A8032A311F96', 'A8032A31204A'] ['A8032A311F96', 'A8032A311F96', 'A8032A31204A']
['A8032A311F96', 'A8032A31204A', 'A8032A311F6A'] ['A8032A311F96', 'A8032A311F6A']


INFO - different mac_module detected at 2023-07-11 13:46:36
INFO - different mac_module detected at 2023-07-11 13:46:58
INFO - different mac_module detected at 2023-07-11 13:47:00
INFO - different mac_module detected at 2023-07-11 13:47:02
INFO - different mac_module detected at 2023-07-11 13:47:38
INFO - different mac_module detected at 2023-07-11 13:47:40
INFO - different mac_module detected at 2023-07-11 13:48:10
INFO - different mac_module detected at 2023-07-11 13:48:12
INFO - different mac_module detected at 2023-07-11 13:48:36
INFO - different mac_module detected at 2023-07-11 13:48:38
INFO - different mac_module detected at 2023-07-11 13:49:26
INFO - different mac_module detected at 2023-07-11 13:49:28
INFO - different mac_module detected at 2023-07-11 13:49:40
INFO - different mac_module detected at 2023-07-11 13:49:42
INFO - different mac_module detected at 2023-07-11 13:53:10
INFO - different mac_module detected at 2023-07-11 13:53:12
INFO - different mac_module detected at 

['A8032A311DAA', 'A8032A311DAA', 'A8032A311F6A'] ['A8032A311F6A', 'A8032A311DAA']


INFO - different mac_module detected at 2023-07-11 13:59:28
INFO - different mac_module detected at 2023-07-11 13:59:30
INFO - different mac_module detected at 2023-07-11 13:59:42
INFO - different mac_module detected at 2023-07-11 13:59:44
INFO - different mac_module detected at 2023-07-11 13:59:56
INFO - different mac_module detected at 2023-07-11 13:59:58
INFO - different mac_module detected at 2023-07-11 14:00:22
INFO - different mac_module detected at 2023-07-11 14:00:24
INFO - different mac_module detected at 2023-07-11 14:01:26
INFO - different mac_module detected at 2023-07-11 14:01:28
INFO - different mac_module detected at 2023-07-11 14:01:46
INFO - different mac_module detected at 2023-07-11 14:01:48
INFO - different mac_module detected at 2023-07-11 14:01:50
INFO - different mac_module detected at 2023-07-11 14:01:54
INFO - different mac_module detected at 2023-07-11 14:01:56
INFO - different mac_module detected at 2023-07-11 14:02:12
INFO - different mac_module detected at 

In [ ]:
for i in time_range:
    current_strongest=get_strongest_rssi(rssi_df,i,timedelta,number_of_mac_module).to_list()
    if len(static_mac_module_list)<number_of_mac_module:
        for j in current_strongest:
            if j not in static_mac_module_list:
                static_mac_module_list.append(j)
        print(static_mac_module_list,current_strongest)

    different_count=0
    for j in current_strongest:
        if j not in static_mac_module_list:
            different_count+=1
 
    if different_count>=2:
        logger.info("more than 2 different mac_module detected at "+str(i))
        logger.info("static mac module set: "+str(current_strongest))
        #reset the static_mac_module_list
        static_mac_module_list=current_strongest
    # elif different_count>=1:
    #     logger.info("different mac_module detected at "+str(i))

